In [ ]:
# default_exp core

# core

> Contains validate_param() and supporting functions.  

**USAGE**

The MVal.validate_param() method is the public facing method of MVal class used for validating variables.  It validates in the following two ways:

  > 1. Checking that parameter is the corret type.
  >     - Supports primative types and objects.
  >     - Supports nested structures (typing module notation used).
  >
  >
  > 2. Checking that parameter meets any required restrictions.  Which can take any of the following forms
  >     - No restrictions -> ex. None
  >     - Bounds expression -> ex. "(0,)"  ***Only supported for `<class 'int'>` and `<class 'float'>` types***
  >     - Set of allowed values -> ex. {1,2,3,4,5}
  >     - Callable function returning True, if meets restrictions, and False otherwise

---
**LIMITATIONS**
- Only supports the following from typing module:
      List, Tuple, Dict, and Union
- Only supports Tuple definitions of fixed size
      i.e. Tuple[int, float, str] and not Tuple[int, ...]
- Currently restrictions are applied to entire value i.e. not nested values

---
**NOTES:**

This package comes with no warranty and no garantees.  

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

from inspect import getmodule
from typing import List, Tuple, Dict, Union, TypeVar, Optional, Callable
import os
import operator

#hide

## MVal

---
**Methods**

Public

  - validate_params()

Private

  - _check_type()
  - _check_restrictions()
  - _validate_bounds_expression()
  - _check_bounds()


In [ ]:
#exports
class MVal:

    def _check_type(p, p_type):
        """
        subclasses always ok for now.

        Operates on
        -----------

        Union, List, Tuple, Dict

        TypeVar is a generic type that doesn't have .__origin__ so any param who's type is supposed to be 
        TypeVar automatically passes.  
        """
        if (getmodule(p_type).__name__ == "typing") and (p_type is not TypeVar):

            parent_type = p_type.__origin__  

            if parent_type is tuple:
                if not issubclass(type(p), parent_type):
                    raise TypeError("expected type {}, but received {}".format(p_type, type(p)))
                inner_types = p_type.__args__
                if len(inner_types) != len(p):
                    raise ValueError(("tuple expected {} elements, but " +
                                      "received {}").format(len(inner_types), len(p)))
                for idx, child in enumerate(p):
                    MVal._check_type(child, inner_types[idx])

            if parent_type is list:
                if not issubclass(type(p), parent_type):
                    raise TypeError("expected type {}, but received {}".format(p_type, type(p)))
                inner_type = p_type.__args__[0]
                for child in p:
                    MVal._check_type(child, inner_type)

            if parent_type is dict:
                if not issubclass(type(p), parent_type):
                    raise TypeError("expected type {}, but received {}".format(p_type, type(p)))
                key_type, value_type = p_type.__args__
                for key in p:
                    MVal._check_type(key, key_type)
                    MVal._check_type(p[key], value_type)

            if parent_type is Union:
                allowed_types = p_type.__args__
                for atype in allowed_types:
                    atype_origin = atype.__origin__ if (getmodule(atype).__name__ == "typing") else atype
                    if issubclass(type(p), atype_origin):
                        MVal._check_type(p, atype)
                        break
                else:
                    raise TypeError(("expected one of the following types {}, " +
                                     "but received {}").format(allowed_types, type(p)))


        if (getmodule(p_type).__name__ != "typing") and not issubclass(type(p), p_type):  
            raise TypeError("expected type {}, but received {}".format(p_type, type(p)))
    
    def _check_restrictions(p, p_restrictions, p_type):
        """
        p_restriction can be:
          None >> no restriction
          string >> specialized to represent upper and lower bounds
          set >> contains allowed values

          List[int] >> represents a list of codes for functions to run
          int >> represents a single code for a function

          function >> if it is callable p_restriction(p) is run result should be bool.  
          True is good, False is bad
        """
        # "(0,100]", {1,2,3,4,5}, 

        # p_restriction is a callable function
        if callable(p_restrictions):
            if not p_restrictions(p):
                raise ValueError(("{} doesn't meet the restrictions enforced by " +
                                  "function: {}").format(p, p_restrictions))

        # p_restriction is a bounds expression
        elif type(p_restrictions) is str:
            MVal._validate_bounds_expression(p_type, p_restrictions)  # ensure bounds expression is valid
            p = MVal._check_bounds(p, p_restrictions)

        # p_restriction is a set of allowed values
        elif type(p_restrictions) is set:
            if p not in p_restrictions:
                raise ValueError(("{} doesn't meet the restrictions enforced by " +
                                  "set of allowed values: {}").format(p, p_restrictions))

        # p_restriction is None (i.e. no restrictions)
        elif p_restrictions is None:
            pass

        # p_restriction is something not recognized
        else:
            raise TypeError(("Bad p_restriction type.  Expected: None, a callable function that " +
                             "returns a bool representing whether the value meets the restrictions, " +
                             "a string representing a bounds expression, or a set of allowed values.  " +
                             "However, received type {}").format(type(p_restrictions)))
        return p

    def _validate_bounds_expression(value_type: type, expression: str) -> str:
        """
        Check bounds expression string to ensure that it is valid for evaluation.  In order to be a valid
        expression the following must be True:

            1) start must be '(' or '['
            2) end must be ')' or ']'
            3) must contain only one ','
            4) value_type must be either float or int
            4) values other than these must be interpretable as either ints or floats whichever value_type is
            5) if upper or lower bound is open (i.e. to +/- infinity) then it should be preceded / followed by
               a curly bracket
        """

        expression = expression.strip() # strip whitespace from start and end of strip

        # 1) check that start is one of '(' or '['
        if expression[0] != '(' and expression[0] != '[':
            raise ValueError(("Bad Expression: {}\nShould start with either " +
                              "'(' or '['.  \n").format(expression))

        # 2) check that end is one of ')' or ']'
        if expression[-1] != ')' and expression[-1] != ']':
            raise ValueError("Bad Expression: {}\nShould end with either ')' or ']'.  \n".format(expression))

        # 3) check that expression contains only one comma
        if expression.count(',') != 1:
            raise ValueError(("Bad Expression: {}\nShould contain exactly one ',' seperating lower and " +
                              "upper bounds.  \n").format(expression))

        # 4) check that value type is either float or int
        if value_type is not int and value_type is not float:
            raise ValueError(("Bad Expression: {}\nOnly supported for primative types: <int> and " +
                              "<float>, not {}\n").format(expression, value_type))

        # 5) check that expression values are interpretable as value_type
        lower_val, upper_val = expression[1:-1].split(',')
        if lower_val != '':  # empty string represents an open bound
            try:
                value_type(lower_val)
            except ValueError as error:
                raise ValueError(("Bad Expression: {}\nLower bound value is not interpretable " +
                                  "as {}.\n").format(expression, value_type))
        if upper_val != '':
            try:
                value_type(upper_val)
            except ValueError as error:
                raise ValueError(("Bad Expression: {}\nUpper bound value is not interpretable " +
                                  "as {}.\n").format(expression, value_type))

        # 6) check that given lower_val / upper_val is '' then it must be preceded / followed by '(' / ')'

        if lower_val == '' and expression[0] != '(':
            raise ValueError(("Bad Expression: {}\nLower bound is open, therefore it must be preceded by " +
                              "'(' not '['.\n").format(expression))
        if upper_val == '' and expression[-1] != ')':
            raise ValueError("Bad Expression: {}\nUpper bound is open, therefore it must be followed by " +
                             "')' not ']'.\n".format(expression))

        # if execution reaches this point all tests above are passing
        return expression

    def _check_bounds(value: Union[int, float], bounds: str) -> Union[int, float]:
        """
        Assumes that the value is the correct type and the bounds have already been validated.

        Check to see that the variable value falls into the bounds represented by value_restrictions.
        value_restrictions is a string of the form: '(lower_bound, upper_bound]'.  A curly bracket means
        everything upto but not including and a square bracket is inclusive.  If a bound is omitted then it is
        treated as open (to infinity) and must be preceded/followed by a curly bracket.  If the value satifies
        the bounds expression then True is returned otherwise False is returned.
        """

        if value is None:
            raise ValueError("Value {} does not fall within the bounds: {}".format(value, bounds))

        lower_bound, upper_bound = bounds.split(',')  # should always only be one ','
        lower_bound, upper_bound = lower_bound.strip(), upper_bound.strip() # strip away any whitespace

        lower_op = operator.gt if lower_bound[0] == '(' else operator.ge
        upper_op = operator.lt if upper_bound[-1] == ')' else operator.le

        lower_val = lower_bound[1:]  # will be an empty string if no value supplied (i.e open bound)
        upper_val = upper_bound[:-1]

        value_type = type(value)

        if lower_val != '':
            # lower_op is going to be operator.ge or operator.gt
            lower_val = value_type(lower_val)
            if not lower_op(value, lower_val):
                raise ValueError("Value {} does not fall within the bounds: {}".format(value, bounds))
        if upper_val != '':
            # upper_op is going to be operator.le or operatory.lt
            upper_val = value_type(upper_val)
            if not upper_op(value, upper_val):
                raise ValueError("Value {} does not fall within the bounds: {}".format(value, bounds))

        # if execution reaches this point value satisfies bounds expression
        return value

    def validate_param(p, p_type, p_name, p_restrictions=None):
        """
        
        """
        # 1) check type
        try:
            MVal._check_type(p, p_type) # recursive function, therefore catch error here for better error msg
        except TypeError as error:
            raise TypeError("{} expected type {}, but received {} of type {}".format(p_name, p_type, p, 
                                                                                     type(p))) from error

        # 2) check restriction
        p = MVal._check_restrictions(p, p_restrictions, p_type)
        return p

## TODO

1. Add concise typing hints to functions
2. Implement more builtin restriction checks (i.e. when certain codes are sent it preforms
   corresponding check).  May want to turn method into classmethods if this is done
3. Add support for regular expressions and come up with way to differentiate them from bounds expression
3. Look into making custom exceptions
4. When dealing with nested structure come up with way to choose if restrictions apply to whole thing or at what
   level to apply.  i.e. add a restriction_level parameter
5. Wrap everything up into a class (and import validate_param in the `__init__.py` file)
    a. make functions other than validate_params private
6. Beef up the documentation
7. Add a parameter allow_None and if the value is None then it is ok and circumvents the checks